In [384]:
import bs4, requests
import pandas as pd
import os
import itertools

In [385]:
file = "C:/Users/tomek/zipline/CME_metadata.csv"
path = 'C:/Users/tomek/zipline/excel_meta/'

In [386]:
f = pd.read_csv(file)
f.drop(f[f['description'].str.contains('Dataset description', regex=False) == True].index, inplace=True)
f['description'] = f['description'].apply(lambda x: x.split('<a href=')[-1].split('>http')[0])
#f['description'] = f['description'].str.replace('contract_specifications', 'product_calendar_futures')
f['year'] = f.code.apply(lambda x: x[-4:])
f = f[f['year'] > '2017']
f['root'] = f.code.apply(lambda x: x[:-5])
f.rename(columns={'code': 'symbol'}, inplace=True)
f['exch_symbol'] = f['symbol'].apply(lambda x: x[:-4] + x[-2:])
f.head()

,symbol,name,description,refreshed_at,from_date,to_date,year,root,exch_symbol
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-02-03 01:07:36,2014-11-21,2018-02-02,2018,0D,0DF18
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-19 00:15:29,2015-02-12,2018-10-18,2019,0D,0DF19
5,0DF2020,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-10-19 00:15:29,2017-12-04,2018-10-18,2020,0D,0DF20
6,0DF2021,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2017-12-07 01:08:51,2017-12-04,2017-12-06,2021,0D,0DF21
11,0DG2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-03-03 01:07:30,2014-11-21,2018-03-02,2018,0D,0DG18


In [462]:
df = pd.DataFrame(f.groupby('root')['description'].apply(lambda x: x.iloc[0]))
df.columns = ['url']
df.head()

,url
root,
0D,http://www.cmegroup.com/trading/energy/refined...
0F,http://www.cmegroup.com/trading/energy/refined...
1N,http://www.cmegroup.com/trading/energy/refined...
1NB,http://www.cmegroup.com/trading/energy/refined...
1R,http://www.cmegroup.com/trading/energy/petroch...


In [9]:
f[f['root'] == 'ES']['description'].iloc[1]

'http://www.cmegroup.com/trading/equity-index/us-index/e-mini-sandp500_contract_specifications.html '

In [388]:
def get_meta(url):
    r = requests.get(url)
    r.raise_for_status()
    soup = bs4.BeautifulSoup(r.text, "html.parser")    
    for e in soup.findAll('br'):
        e.extract()
    tds = soup('td', attrs={'class': 'prodSpecAtribute'})
    data = {}
    for t in tds:
        data[t.parent('td')[0].contents[0].strip()] = t.parent('td')[1].contents
    for k, v in data.items():
        new_value = ''
        for c in v:
            if c.string is None:
                continue
            new_value += c.string.strip() + ' '
        data[k] = new_value
    return data

In [389]:
dics = {}
missing_roots = []
for row in df.iterrows():
    try:
        dics[row[0]] = get_meta(row[1].item())
    except:
        missing_roots.append(row[0])

In [390]:
len(missing_roots)

18

In [391]:
pd.DataFrame.from_dict(dics, orient='index').columns

Index(['Settlement Procedures', 'Product Code', 'Settlement Method',
       'Block Minimum', 'Trading Hours', 'Vendor Codes', 'Listed Contracts',
       'Price Limit Or Circuit', 'Grade And Quality', 'Price Quotation',
       'Minimum Price Fluctuation', 'Contract Unit', 'Termination Of Trading',
       'Position Limits', 'Last Delivery Date', 'Exchange Rulebook',
       'Floating Price', 'Days Or Hours',
       'Trade At Marker Or Trade At Settlement Rules', 'Delivery Procedure',
       'Delivery Period', 'All Or None Minimum', 'Reportable Positions'],
      dtype='object')

In [463]:
results = pd.DataFrame.from_dict(dics, orient='index')[['Product Code', 'Settlement Method', 'Price Quotation',
                                              'Contract Unit', 'Minimum Price Fluctuation', 'Termination Of Trading', 
                                                  'Trading Hours']]

In [464]:
results.reset_index(inplace=True)

In [465]:
results.rename(columns={'index': 'root'}, inplace=True)

In [466]:
results.head()

,root,Product Code,Settlement Method,Price Quotation,Contract Unit,Minimum Price Fluctuation,Termination Of Trading,Trading Hours
0,0D,CME Globex: A0DCME ClearPort: 0DClearing: 0D,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
1,0F,CME Globex: A0FCME ClearPort: 0FClearing: 0F,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
2,1N,CME Globex: N1BCME ClearPort: 1NClearing: 1N,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
3,1NB,CME Globex: D1NCME ClearPort: 1NBClearing: 1NB,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
4,1R,CME Globex: A1RCME ClearPort: 1RClearing: 1R,Financially Settled,U.S. dollars and cents per gallon,"42,000 gallons",$0.00001 per gallon,Trading shall cease on the last business day o...,CME Globex:


In [467]:
df.reset_index(inplace=True)

In [468]:
df.head()

,root,url
0,0D,http://www.cmegroup.com/trading/energy/refined...
1,0F,http://www.cmegroup.com/trading/energy/refined...
2,1N,http://www.cmegroup.com/trading/energy/refined...
3,1NB,http://www.cmegroup.com/trading/energy/refined...
4,1R,http://www.cmegroup.com/trading/energy/petroch...


In [469]:
df = df.merge(results, on='root', how='left')

In [470]:
df.head()

,root,url,Product Code,Settlement Method,Price Quotation,Contract Unit,Minimum Price Fluctuation,Termination Of Trading,Trading Hours
0,0D,http://www.cmegroup.com/trading/energy/refined...,CME Globex: A0DCME ClearPort: 0DClearing: 0D,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
1,0F,http://www.cmegroup.com/trading/energy/refined...,CME Globex: A0FCME ClearPort: 0FClearing: 0F,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
2,1N,http://www.cmegroup.com/trading/energy/refined...,CME Globex: N1BCME ClearPort: 1NClearing: 1N,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
3,1NB,http://www.cmegroup.com/trading/energy/refined...,CME Globex: D1NCME ClearPort: 1NBClearing: 1NB,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
4,1R,http://www.cmegroup.com/trading/energy/petroch...,CME Globex: A1RCME ClearPort: 1RClearing: 1R,Financially Settled,U.S. dollars and cents per gallon,"42,000 gallons",$0.00001 per gallon,Trading shall cease on the last business day o...,CME Globex:


In [471]:
df.to_csv('CME_downloaded_data.csv')

In [472]:
quandl_file = pd.read_csv('CMEGroup_merged.csv')
quandl_file

,root_symbol,root_symbol.1,exchange,name_x,name_y,multiplier,tick_size
0,0D,0D,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.01
1,0F,0F,NYMEX,Mini Singapore Fuel Oil 180 cst (Platts) Futures,NaN,NaN,NaN
2,1N,1N,NYMEX,Singapore Mogas 92 Unleaded (Platts) Futures,NaN,NaN,NaN
3,1NB,1NB,NYMEX,Singapore Mogas 92 Unleaded (Platts) Brent Cra...,NaN,NaN,NaN
4,1R,1R,NYMEX,Propane Non-LDH Mont Belvieu (OPIS) Futures,NaN,NaN,NaN
5,1U,1U,NYMEX,NY Jet Fuel (Platts) vs. NY Harbor ULSD Futures,NaN,NaN,NaN
6,46,46,NYMEX,PJM METED Zone Off-Peak Calendar-Month Day-Ahe...,NaN,NaN,NaN
7,47,47,NYMEX,PJM METED Zone Peak Calendar-Month Day-Ahead L...,NaN,NaN,NaN
8,4L,4L,NYMEX,NYISO Zone F 5 MW Peak Calendar-Month Day-Ahea...,NaN,NaN,NaN
9,4M,4M,NYMEX,NYISO Zone F 5 MW Off-Peak Calendar-Month Day-...,NaN,NaN,NaN


In [473]:
quandl_file.drop('root_symbol.1', axis=1, inplace=True)
quandl_file.rename(columns={'root_symbol': 'root'}, inplace=True)
quandl_file.head()

,root,exchange,name_x,name_y,multiplier,tick_size
0,0D,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.01
1,0F,NYMEX,Mini Singapore Fuel Oil 180 cst (Platts) Futures,NaN,NaN,NaN
2,1N,NYMEX,Singapore Mogas 92 Unleaded (Platts) Futures,NaN,NaN,NaN
3,1NB,NYMEX,Singapore Mogas 92 Unleaded (Platts) Brent Cra...,NaN,NaN,NaN
4,1R,NYMEX,Propane Non-LDH Mont Belvieu (OPIS) Futures,NaN,NaN,NaN


In [474]:
meta = quandl_file.merge(df, on='root', how='left')

In [475]:
meta.to_csv('meta.csv', encoding='utf-8')

In [476]:
meta.head()

,root,exchange,name_x,name_y,multiplier,tick_size,url,Product Code,Settlement Method,Price Quotation,Contract Unit,Minimum Price Fluctuation,Termination Of Trading,Trading Hours
0,0D,NYMEX,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,100.0,0.01,http://www.cmegroup.com/trading/energy/refined...,CME Globex: A0DCME ClearPort: 0DClearing: 0D,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
1,0F,NYMEX,Mini Singapore Fuel Oil 180 cst (Platts) Futures,NaN,NaN,NaN,http://www.cmegroup.com/trading/energy/refined...,CME Globex: A0FCME ClearPort: 0FClearing: 0F,Financially Settled,U.S. dollars and cents per metric ton,100 metric tons,$0.001 per metric ton,Trading shall cease on the last business day o...,CME Globex:
2,1N,NYMEX,Singapore Mogas 92 Unleaded (Platts) Futures,NaN,NaN,NaN,http://www.cmegroup.com/trading/energy/refined...,CME Globex: N1BCME ClearPort: 1NClearing: 1N,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
3,1NB,NYMEX,Singapore Mogas 92 Unleaded (Platts) Brent Cra...,NaN,NaN,NaN,http://www.cmegroup.com/trading/energy/refined...,CME Globex: D1NCME ClearPort: 1NBClearing: 1NB,Financially Settled,U.S. dollars and cents per barrel,"1,000 barrels",$0.001 per barrel,Trading shall cease on the last business day o...,CME Globex:
4,1R,NYMEX,Propane Non-LDH Mont Belvieu (OPIS) Futures,NaN,NaN,NaN,http://www.cmegroup.com/trading/energy/petroch...,CME Globex: A1RCME ClearPort: 1RClearing: 1R,Financially Settled,U.S. dollars and cents per gallon,"42,000 gallons",$0.00001 per gallon,Trading shall cease on the last business day o...,CME Globex:


In [483]:
sectors = pd.read_excel('Product Slate Export.xlsx', header=4, usecols=['Globex', 'Product Name', 'Product Group', 'Sub Group'])

In [484]:
sectors.head()

,Globex,Product Name,Product Group,Sub Group
0,GE,Eurodollar Futures,Interest Rate,Stirs
1,ZF,5-Year T-Note Futures,Interest Rate,US Treasury
2,ZN,10-Year T-Note Futures,Interest Rate,US Treasury
3,ES,E-mini S&P 500 Futures,Equities,US Index
4,ZT,2-Year T-Note Futures,Interest Rate,US Treasury


In [485]:
sectors.rename(columns={'Globex': 'root'}, inplace=True)

In [486]:
sectors.head()

,root,Product Name,Product Group,Sub Group
0,GE,Eurodollar Futures,Interest Rate,Stirs
1,ZF,5-Year T-Note Futures,Interest Rate,US Treasury
2,ZN,10-Year T-Note Futures,Interest Rate,US Treasury
3,ES,E-mini S&P 500 Futures,Equities,US Index
4,ZT,2-Year T-Note Futures,Interest Rate,US Treasury


In [488]:
meta = meta.merge(sectors, on='root', how='left')

In [489]:
meta.to_csv('meta.csv', encoding='utf-8')

In [410]:
args = ['ESH2019', 'http://www.cmegroup.com/trading/equity-index/us-index/e-mini-sandp500_contract_specifications.html']

In [411]:
r = requests.get(args[1])
r.raise_for_status()
soup = bs4.BeautifulSoup(r.text, "html.parser")    

In [412]:
tds = soup('td', attrs={'class': 'prodSpecAtribute'})
data = {}
for t in tds:
    data[t.parent('td')[0].contents[0].strip()] = t.parent('td')[1].contents
    #print(t.parent('td')[0].contents[0].strip(), t.parent('td')[1].contents)

In [413]:
for k, v in data.items():
    new_value = ''
    for c in v:
        #if c.string is None:
            #continue
        print(k)
        print(c)
        print()
        #new_value += c.string.strip() + ' '
    #data[k] = new_value

Settlement Procedures
<a href="http://www.cmegroup.com/confluence/display/EPICSANDBOX/Standard+and+Poors+500+Futures" target="_blank">Settlement Procedures</a>

Product Code

					CME Globex: ES

Settlement Method
Financially Settled

Trading Hours
CME Globex: Sunday - Friday 6:00 p.m. - 5:00 p.m. Eastern Time (ET) with  trading halt 4:15 p.m. - 4:30 p.m.

Trading Hours
<br>
<br>
BTIC: Sunday - Friday 6:00 p.m. - 4:00 p.m. ET<br/>
<br/>
Clearport: Sunday - Friday 6:00 p.m. - 5:00 p.m. ET<br/>
 <br/>
TACO on CME Globex: Sunday - Friday 6:00 p.m. - 9:30 a.m. ET. Monday - Thursday 11:00 a.m. - 5:00 p.m. ET; no 11:00 a.m.- 5:00 p.m. ET session on Friday. Monday - Thursday 5:00 p.m. - 6:00 p.m. ET daily maintenance period.<br/>
 <br/>
TACO on Clearport:  Sunday 6:00 p.m. - Monday 9:30 a.m. ET. Monday - Thursday 11:00 a.m. - 5:00 p.m. ET and 6:00 p.m. - 9:30 a.m. ET. Friday 11:00 a.m. - 5:00 p.m.  ET. Monday - Thursday 5:00 p.m. - 6:00 p.m. ET daily maintenance period.</br></br>

Vendor Code

In [414]:
data

{'Contract Unit': ['$50 x S&P 500 Index'],
 'Exchange Rulebook': [<a href="http://www.cmegroup.com/rulebook/CME/IV/350/358/358.pdf" target="_blank">CME 358</a>],
 'Listed Contracts': ['\r\n\t\t\t\t\t\r\n\t\t\t\t\tFive months in the March Quarterly Cycle (Mar, Jun, Sep, Dec)\r\n\t\t\t\t\t\r\n\t\t\t\t\t'],
 'Minimum Price Fluctuation': ['\r\n\t\t\t\t\t\tOutright: 0.25 index points=$12.50',
  <br/>,
  '\nCalendar Spread: 0.05 index points=$2.50',
  <br/>,
  '\nBTIC: 0.05 index points=$2.50',
  <br/>,
  '\nTACO (ESQ): 0.05 index points=$2.50\r\n\t\t\t\t\t\t'],
 'Position Limits': [<a href="http://www.cmegroup.com/rulebook/files/position-limits-cme.xlsx" target="_blank">CME Position Limits</a>],
 'Price Limit Or Circuit': [<a href="http://www.cmegroup.com/trading/equity-index/price-limit-guide.html" target="_blank">Price Limits</a>],
 'Product Code': ['\r\n\t\t\t\t\tCME Globex: ES'],
 'Settlement Method': ['Financially Settled'],
 'Settlement Procedures': [<a href="http://www.cmegroup.com/c

In [244]:
for k, v in data.items():
    print(k)
    print(v)
    print()

Settlement Procedures
Settlement Procedures 

Product Code
CME Globex: ES 

Settlement Method
Financially Settled 

Trading Hours
CME Globex: Sunday - Friday 6:00 p.m. - 5:00 p.m. Eastern Time (ET) with  trading halt 4:15 p.m. - 4:30 p.m. 

Vendor Codes
Quote Vendor Symbols Listing 

Price Limit Or Circuit
Price Limits 

Listed Contracts
Five months in the March Quarterly Cycle (Mar, Jun, Sep, Dec) 

Minimum Price Fluctuation
Outright: 0.25 index points=$12.50 Calendar Spread: 0.05 index points=$2.50 BTIC: 0.05 index points=$2.50 TACO (ESQ): 0.05 index points=$2.50 

Contract Unit
$50 x S&P 500 Index 

Termination Of Trading
Trading can occur up to 9:30 a.m. Eastern Time (ET) on the 3rd Friday of the contract month  BTIC trading terminates at 4:00 p.m. Eastern Time (ET) on the Thursday before the 3rd Friday of contract month  TACO trading terminates at 9:30 a.m. ET on the Thursday before the 3rd Friday of the contract month 

Position Limits
CME Position Limits 

Exchange Rulebook
CME 